In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### 점수 출력 함수(선생님꺼임 나중에 수정하자)

In [46]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [47]:
df = pd.read_csv('heart_2020_cleaned_preprocessing.csv')
x = df.iloc[:,1:]
y = df['HeartDisease']

### train, test set 분리

In [48]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42 , stratify=y)

### 스케일링 처리

In [36]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#x_train_scaled = scaler.fit(x_train_over).transform(x_train_over)
#x_test_scaled = scaler.transform(x_test)
#pd.DataFrame(x_train_scaled,columns = x.columns)

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,-0.808704,1.146082,-0.20203,-0.208939,2.470658,3.026561,2.105101,-0.949088,-0.800230,0.471101,-0.450776,-1.398732,-0.821200,-1.386787,3.029534,-0.193196,-0.313648
1,-0.993418,-0.872538,-0.20203,-0.208939,-0.438470,-0.493142,-0.475037,1.053643,-2.002365,0.471101,-0.450776,0.714933,1.351501,0.580893,-0.330084,-0.193196,-0.313648
2,0.690737,1.146082,-0.20203,-0.208939,1.467510,3.026561,-0.475037,-0.949088,-1.701831,-1.235935,-0.450776,-1.398732,-0.821200,-0.075000,-0.330084,-0.193196,-0.313648
3,-0.600707,1.146082,-0.20203,-0.208939,-0.538785,-0.493142,-0.475037,-0.949088,0.401904,0.471101,-0.450776,0.714933,1.351501,0.580893,-0.330084,-0.193196,-0.313648
4,-0.245249,1.146082,-0.20203,-0.208939,-0.538785,-0.493142,-0.475037,-0.949088,-0.199163,0.471101,-0.450776,0.714933,1.351501,-1.386787,-0.330084,-0.193196,-0.313648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467871,0.505738,-0.872538,-0.20203,-0.208939,-0.538785,-0.493142,-0.475037,-0.949088,-0.199163,0.471101,-0.450776,0.714933,1.351501,-0.075000,-0.330084,-0.193196,-0.313648
467872,-1.567155,1.146082,-0.20203,-0.208939,2.470658,3.026561,2.105101,-0.949088,1.303504,0.471101,-0.450776,-1.398732,-0.096966,0.580893,-0.330084,-0.193196,-0.313648
467873,0.307398,1.146082,-0.20203,-0.208939,0.965937,-0.493142,2.105101,-0.949088,0.702437,0.471101,1.661317,-1.398732,0.627267,-0.730893,-0.330084,-0.193196,-0.313648
467874,0.154793,1.146082,-0.20203,-0.208939,0.026842,0.130931,2.105101,1.053643,0.702437,0.471101,-0.450776,-1.398732,-0.821200,-0.730893,-0.330084,-0.193196,-0.313648


-------------------------------------------------------

In [49]:
print(y_train.value_counts(normalize=True))

0    0.914406
1    0.085594
Name: HeartDisease, dtype: float64


### 위 분포를 보면 심장병인 사람과 아닌 사람의 비율이 처참한 것을 볼 수 있음. 
### SMOTE 를 사용해 분포를 맞춰준다

In [51]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_train_over , y_train_over = smote.fit_resample(x_train,y_train)
print(y_train_over.value_counts(normalize=True))
len(x), len(x_train_over) # 데이터가 150000개 가량 증가한 것을 볼 수 있음.

0    0.5
1    0.5
Name: HeartDisease, dtype: float64


(319795, 467876)

In [53]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class = 'auto', solver = 'lbfgs', random_state=42)
reg = lr.fit(x_train_over,y_train_over)
y_pred = reg.predict(x_test)
get_clf_eval(y_test,y_pred,reg.predict_proba(x_test)[:,1])

오차 행렬
[[40601 17883]
 [ 1717  3758]]
정확도: 0.6936, 정밀도: 0.1737, 재현율: 0.6864,    F1: 0.2772, AUC:0.7543


# 우리 데이터는 재현율이 높은것이 중요함
### 심장병에 걸릴사람에게 걸리지 않을거야 라고 말하면 안되기 때문